In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from astropy.table import Table
import gravity_obs.p2_tools as p2t

In [3]:
table_path = '../../gvprep/tabs/star_k9-12mag_grp10-12mag_gaia_wide_pairs.ipac'
tb = Table.read(table_path, format='ascii.ipac')

In [4]:
tb

name,ra_2000_hms,dec_2000_dms,ra_2000,dec_2000,parallax,pm,pmra,pmdec,radial_velocity,Grp,G,ra_2016,dec_2016,source_id,H,K,name_ft,ra_2000_hms_ft,dec_2000_dms_ft,ra_2000_ft,dec_2000_ft,H_ft,K_ft,parallax_ft,pm_ft,pmra_ft,pmdec_ft,radial_velocity_ft,Grp_ft,G_ft,ra_2016_ft,dec_2016_ft,source_id_ft,sobj_x,sobj_y,sep,pa
,,,,,mas,mas / yr,mas / yr,mas / yr,km / s,mag,mag,deg,deg,,mag,mag,,,,,,mag,mag,mas,mas / yr,mas / yr,mas / yr,km / s,mag,mag,deg,deg,,mas,mas,arcsec,deg
str28,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,str28,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64
Gaia DR3 2307957683607394944,00:00:07.774,-36:59:26.190,0.03238982385615093,-36.99060822111795,1.1767,27.479,5.026,-27.015,82.27,13.842175,14.34074,0.03241779066,-36.99072828913,2307957683607394944,12.957,12.94,Gaia DR3 2307957649246804480,00:00:06.930,-36:59:38.769,0.028875122687,-36.99410249,10.704000473,10.645000458,2.9988,17.12,17.046,1.591,8.1,11.463291,11.925517,0.02896997595,-36.994095421,2307957649246804480,9796.040053146273,11840.861285766401,15.367771364502,39.60122864468628
Gaia DR3 2421121924304078976,00:00:16.920,-12:55:08.765,0.07050151402408138,-12.919101440137313,2.2326,27.13,26.933,3.265,-11.96,11.703443,12.195138,0.07062432363,-12.91908693094,2421121924304078976,10.876,10.803,Gaia DR3 2421121928599635200,00:00:16.694,-12:55:13.214,0.069558668476,-12.920337332,10.19299984,10.151000023,2.2798,27.605,27.416,3.223,--,10.90099,11.327644,0.06968368445,-12.920323009,2421121928599635200,3295.84322528464,4450.28702271995,5.537836865622196,36.52332103687524
Gaia DR3 2421111070922287104,00:00:22.839,-13:03:32.861,0.09516438288292063,-13.059128157632033,3.979,16.213,15.78,3.72,--,14.776588,15.835312,0.09523637877,-13.05911162454,2421111070922287104,12.78,12.562,Gaia DR3 2421111070920983936,00:00:21.500,-13:03:19.478,0.08958214975,-13.05541053,10.18500042,10.072999954,3.9805,16.535,16.534,0.174,-10.75,11.216269,11.779585,0.08965758473,-13.055409758,2421111070920983936,19556.850313346084,-13292.14661883927,23.646385683976,124.20265508900314
Gaia DR3 2306294775349638016,00:00:30.857,-39:37:55.848,0.1285726731366147,-39.63218009989112,2.0678,13.998,10.162,-9.626,-34.27,12.219716,12.713209,0.12863131809,-39.63222288327,2306294775349638016,11.343,11.237,Gaia DR3 2306294844069169152,00:00:30.020,-39:37:29.142,0.12508212083,-39.624761691,10.102000237,10.083000183,1.6175,6.533,0.233,-6.529,--,10.663338,10.898009,0.1250834656,-39.62479071,2306294844069169152,9934.026971088353,-26786.404962517583,28.56915089179757,159.65215827455262
Gaia DR3 2313537739478102272,00:00:33.155,-32:45:10.266,0.13814580434338805,-32.75285178320055,6.7574,38.6,38.578,1.315,--,14.971152,16.133118,0.13834967432,-32.75284593942,2313537739478102272,12.822,12.577,Gaia DR3 2313537739478102144,00:00:34.229,-32:44:49.699,0.14261896387,-32.747138685,10.201000214,10.031999588,6.3807,39.37,39.29,2.518,2.47,11.864354,12.706331,0.14282658239,-32.747127497,2313537739478102144,-13561.443352500093,-20598.481477314373,24.66191770674642,213.35974895543876
Gaia DR3 2421159861750790912,00:00:41.743,-12:46:05.577,0.17392725467051767,-12.768215741491314,2.4376,5.042,-0.012,5.042,--,14.638123,15.733018,0.17392720206,-12.76819333432,2421159861750790912,12.508,12.216,Gaia DR3 2421159861750791168,00:00:40.673,-12:46:16.976,0.16947134235,-12.771382222,10.666999817,10.574999809,3.2877,25.196,21.829,-12.583,14.15,11.448132,11.924252,0.16957082173,-12.771438148,2421159861750791168,15080.891331573466,11854.11381473115,19.18210879145793,51.831400954742605
Gaia DR3 2420737821084438400,00:00:47.728,-13:57:47.147,0.19886624215817017,-13.963096382051058,2.9591,20.171,19.919,3.178,-6.88,13.112193,13.765867,0.1989574685,-13.96308225563,2420737821084438400,11.885,11.743,Gaia DR3 2420737816788215680,00:00:47.870,-13:58:10.258,0.19945858198,-13.969

In [5]:
tb.add_index('name')

In [29]:
from gravity_obs import ob_add_target
def dual_wide_for_target(grav, target, folder_name=None,
                         usename=False, verify=False):
    ao_mode = "LGS_VIS"
    tel_alpha = target['ra_2000_hms']
    tel_delta = target['dec_2000_dms']

    if usename:
        obname = f"{ao_mode}_{target['name']}".replace(' ', '_')
    else:
        obname = f"{ao_mode}_{tel_alpha.split('.')[0].replace(':', '')}{tel_delta.split('.')[0].replace(':', '')}_{target['Grp_ft']:4.1f}_{target['K_ft']:4.1f}_{target['K']:4.1f}".replace(' ', '_')


    ob, obV = grav.create_OB(obname, folder_name=folder_name, overwrite=True)
    obsDes = ob['obsDescription']
    obsDes['name'] = target['name']
    obsDes['userComments'] = 'Dual on-axis observation test'

    ob_add_target(ob, target['name'], 
                    ra = target['ra_2000_hms'], 
                    dec = target['dec_2000_dms'],
                    pma = target['pmra'],
                    pmd = target['pmdec'])
    ob, obV = grav.save_OB(obname, folder_name=folder_name)


    dit, ndit = 3, 40
    grav.add_GRAVITY_dual_wide_acq(name=obname, folder_name=folder_name,
                                   met_mode = 'OFF',
                                   sc_name = target['name'],
                                   sc_kmag = target['K'],
                                   sc_hmag = target['H'],
                                   sc_plx = target['parallax']/1e3,
                                   ft_name = target['name_ft'],
                                   ft_alpha = target['ra_2000_hms_ft'],
                                   ft_delta = target['dec_2000_dms_ft'],
                                   ft_kmag = target['K_ft'],
                                   ft_hmag = target['H_ft'],
                                   ft_plx = target['parallax_ft']/1e3,
                                   ngs_mag = target['Grp_ft'],
                                   ao_mode = ao_mode,
                                   baseline = ['UTs'])
    grav.add_GRAVITY_dual_obs_exp(obname, folder_name=folder_name,
                                    dit=dit, ndit_obj=ndit, ndit_sky=ndit,
                                    sky_x=8000, sky_y=8000)

    # optional verification
    if verify:
        obID = ob['obId']
        response, _ = grav.api.verifyOB(obID,True)
        if response['observable']:
            print('*** Congratulations. Your OB' , obID, ob['name'], 'is observable!')
        else:
            print('OB', obID, 'is >>not observable<<. See messages below.')
            print(' ', '\n  '.join(response['messages']))

In [30]:
grav = p2t.p2api_GRAVITY('116.29H7.001', 'GPLUSTeam', 'GPAOwFRINGES', no_warning=True)
root_folder = 'TTR-116.0046 October First Light'
grav.set_rootFolder(root_folder)

In [31]:
folder_name = 'dual_wide'
try:
    grav.get_folder(folder_name)
except Exception:
    grav.create_folder(folder_name, overwrite=True)

In [33]:
#target_names = ["Gaia DR3 2314790972280620288"]
target_names = ["Gaia DR3 2307993967491185280"]
for obj_name in target_names:
    target = tb.loc[obj_name]
    dual_wide_for_target(grav, target, folder_name=folder_name)

In [34]:
target

name,ra_2000_hms,dec_2000_dms,ra_2000,dec_2000,parallax,pm,pmra,pmdec,radial_velocity,Grp,G,ra_2016,dec_2016,source_id,H,K,name_ft,ra_2000_hms_ft,dec_2000_dms_ft,ra_2000_ft,dec_2000_ft,H_ft,K_ft,parallax_ft,pm_ft,pmra_ft,pmdec_ft,radial_velocity_ft,Grp_ft,G_ft,ra_2016_ft,dec_2016_ft,source_id_ft,sobj_x,sobj_y,sep,pa
,,,,,mas,mas / yr,mas / yr,mas / yr,km / s,mag,mag,deg,deg,,mag,mag,,,,,,mag,mag,mas,mas / yr,mas / yr,mas / yr,km / s,mag,mag,deg,deg,,mas,mas,arcsec,deg
str28,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,str28,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64
Gaia DR3 2307993967491185280,00:01:00.455,-36:39:32.549,0.2518976080315839,-36.65904144409833,2.9163,23.439,-12.92,-19.557,19.79,12.058623,12.557637,0.25182602822,-36.65912836386,2307993967491185280,11.175,11.104,Gaia DR3 2307993757037112320,00:01:01.829,-36:39:46.246,0.25762219672,-36.662846014,10.581999779,10.517999649,2.936,23.379,-13.201,-19.295,20.28,11.62263,12.188298,0.25754905425,-36.662931769,2307993757037112320,-16524.90287035847,13689.19546090158,21.458482873715703,309.638280528905
